In [6]:
class Node:
    """Lightweight, nonpublic class for storing a tree node."""
    __slots__ = '_element', '_parent', '_left', '_right'

    def __init__(self, element, parent = None, left = None, right = None):
        self._element = element
        self._parent = parent
        self._left = left
        self._right = right

    def element(self):
        """Return the element stored at this Node."""
        return self._element

In [7]:
import import_ipynb
import collections

from linked_lists import LinkedQueue # type: ignore because its been imported by import_ipynb

class Tree:
  """Abstract base class representing a tree structure."""

  # ---------- abstract methods that concrete subclass must support ----------
  def root(self):
    """Return Node representing the tree's root (or None if empty)."""
    raise NotImplementedError('must be implemented by subclass')

  def parent(self, n):
    """Return Node representing n's parent (or None if n is root)."""
    raise NotImplementedError('must be implemented by subclass')

  def num_children(self, n):
    """Return the number of children that Node n has."""
    raise NotImplementedError('must be implemented by subclass')

  def children(self, n):
    """Generate an iteration of Nodes representing n's children."""
    raise NotImplementedError('must be implemented by subclass')

  def __len__(self):
    """Return the total number of elements in the tree."""
    raise NotImplementedError('must be implemented by subclass')

  # ---------- concrete methods implemented in this class ----------
  def is_root(self, n):
    """Return True if Node n represents the root of the tree."""
    return self.root() == n

  def is_leaf(self, n):
    """Return True if Node n does not have any children."""
    return self.num_children(n) == 0

  def is_empty(self):
    """Return True if the tree is empty."""
    return len(self) == 0

  def depth(self, n):
    """Return the number of levels separating Node n from the root."""
    if self.is_root(n):
      return 0
    else:
      return 1 + self.depth(self.parent(n))

  def _internal_recursive_height(self, n):
    """Return the height of the subtree rooted at Node n."""
    if self.is_leaf(n):
      return 0
    else:
      return 1 + max(self._internal_recursive_height(c) for c in self.children(n))

  def height(self, n=None):
    """Return the height of the subtree rooted at Node n.

    If n is None, return the height of the entire tree.
    """
    if n is None:
      n = self.root()
    return self._internal_recursive_height(n)

  def __iter__(self):
    """Generate an iteration of the tree's elements."""
    for n in self.nodes():
      yield n.element()

  def nodes(self):
    """Generate an iteration of the tree's nodes."""
    return self.preorder()

  def preorder(self):
    """Generate a preorder iteration of Nodes in the tree."""
    if not self.is_empty():
      for n in self._subtree_preorder(self.root()):
        yield n

  def _subtree_preorder(self, n):
    """Generate a preorder iteration of Nodes in subtree rooted at n."""
    yield n
    for c in self.children(n):
      for other in self._subtree_preorder(c):
        yield other

  def postorder(self):
    """Generate a postorder iteration of Nodes in the tree."""
    if not self.is_empty():
      for n in self._subtree_postorder(self.root()):
        yield n

  def _subtree_postorder(self, n):
    """Generate a postorder iteration of Nodes in subtree rooted at n."""
    for c in self.children(n):
      for other in self._subtree_postorder(c):
        yield other
    yield n

  def breadthfirst(self):
    """Generate a breadth-first iteration of the Nodes of the tree."""
    if not self.is_empty():
      fringe = LinkedQueue()
      fringe.enqueue(self.root())
      while not fringe.is_empty():
        n = fringe.dequeue()
        yield n
        for c in self.children(n):
          fringe.enqueue(c)

In [8]:
class BinaryTree(Tree):
  """Abstract base class representing a binary tree structure."""

  # --------------------- additional abstract methods ---------------------
  def left(self, n):
    """Return a Node representing n's left child.

    Return None if n does not have a left child.
    """
    raise NotImplementedError('must be implemented by subclass')

  def right(self, n):
    """Return a Node representing n's right child.

    Return None if n does not have a right child.
    """
    raise NotImplementedError('must be implemented by subclass')

  # ---------- concrete methods implemented in this class ----------
  def sibling(self, n):
    """Return a Node representing n's sibling (or None if no sibling)."""
    parent = self.parent(n)
    if parent is None:
      return None
    else:
      if n == self.left(parent):
        return self.right(parent)
      else:
        return self.left(parent)

  def children(self, n):
    """Generate an iteration of Nodes representing n's children."""
    if n._left is not None:
        yield n._left
    if n._right is not None:
        yield n._right

  def inorder(self):
    """Generate an inorder iteration of Nodes in the tree."""
    if not self.is_empty():
      for n in self._subtree_inorder(self.root()):
        yield n

  def _subtree_inorder(self, n):
    """Generate an inorder iteration of Nodes in subtree rooted at n."""
    if self.left(n) is not None:
      for other in self._subtree_inorder(self.left(n)):
        yield other
    yield n
    if self.right(n) is not None:
      for other in self._subtree_inorder(self.right(n)):
        yield other

  # override inherited version to make inorder the default
  def nodes(self):
    """Generate an iteration of the tree's Nodes."""
    return self.inorder()

### Exercise one
" Implement a version of a Binary Tree that doesn’t use the Position structure "

In [ ]:
class NodeBinaryTree(BinaryTree):
  """Abstract class representing a binary tree with the use of Nodes instead of Positions."""
  __slots__ = '_root', '_size'

  def __init__(self, root=None, size=0):
      self._root = root
      self._size = size

  def _add_root(self, e):
    """Place element e at the root of an empty tree and return it.

    Raise ValueError if tree nonempty.
    """
    if self._root is not None:
      raise ValueError('Root exists')
    self._size = 1
    self._root = Node(e)
    return self._root

  def _add_left(self, n, e):
    """Create a new left child for the parent Node n, storing element e.

    Return the new Node.
    Raise ValueError if n already has a left child.
    """
    if n._left is not None:
      raise ValueError('Left child exists')
    self._size += 1
    n._left = Node(e, n)
    return n._left

  def _add_right(self, p, e):
    """Create a new right child for the parent Node p, storing element e.

    Return the new Node.
    Raise ValueError if p already has a right child.
    """
    if p._right is not None:
      raise ValueError('Right child exists')
    self._size += 1
    p._right = Node(e, p)
    return p._right

  def _replace(self, n, e):
    """Replace the element at origin node n with e, and return old element."""
    old = n._element
    n._element = e
    return old
  
  
  def _delete(self, n):
    """Delete the node n, and replace it with its child, if any.

    Return the element that had been stored at Position p.
    Raise ValueError if Position p is invalid or p has two children.
    """
    if self.num_children(n) == 2:
      raise ValueError('Position has two children')
    child = n._left if n._left else n._right
    if child is not None:
      child._parent = n._parent
    if n is self._root:
      self._root = child
    else:
      parent = n._parent
      if n is parent._left:
        parent._left = child
      else:
        parent._right = child
    self._size -= 1
    n._parent = n
    return n._element
  
  def _attach(self, n, t1, t2):
    """Attach trees t1 and t2, respectively, as the left and right subtrees of the external Node n.

    As a side effect, set t1 and t2 to empty.
    Raise ValueError if Node n is not external.
    Raise TypeError if trees t1 and t2 do not match type of this tree.
    """
    if not self.is_leaf(n):
      raise ValueError('position must be leaf')
    if not type(self) is type(t1) is type(t2):
      raise TypeError('Tree types must match')
    self._size += len(t1) + len(t2)
    if not t1.is_empty():
      t1._root._parent = n
      n._left = t1._root
      t1._root = None
      t1._size = 0
    if not t2.is_empty():
      t2._root._parent = n
      n._right = t2._root
      t2._root = None
      t2._size = 0

  # Implementing Tree class abstract functions
  def root(self):
    """Return Node representing the tree's root (or None if empty)."""
    if self.is_empty():
      return None
    return self._root

  def parent(self, n):
    """Return Node representing n's parent (or None if n is root)."""
    if self.is_root(n):
      return None
    return n._parent

  def num_children(self, n):
    """Return the number of children that Node n has."""
    count = 0
    if n._left is not None:
      count += 1
    if n._right is not None:
      count += 1
    
    return count

  def __len__(self):
    """Return the total number of elements in the tree."""
    return self._size

  # Implementing BinaryTree class abstract functions
  def left(self, n):
    """Return a Node representing n's left child.

    Return None if n does not have a left child.
    """
    return n._left

  def right(self, n):
    """Return a Node representing n's right child.

    Return None if n does not have a right child.
    """
    return n._right
  
  

In [22]:
node_binary_tree = NodeBinaryTree()
node_binary_tree._add_root(4)
node_binary_tree._add_left(node_binary_tree.root(), 3)
node_binary_tree._add_right(node_binary_tree.root(), 5)
node_binary_tree._add_left(node_binary_tree.root()._left, 2)
node_binary_tree._add_right(node_binary_tree.root()._right, 6)
for c in node_binary_tree.breadthfirst():
    print(c._element)


4
3
5
2
6


### Exercise two
" Implement the method \_\_str\_\_ and \_\_repr\_\_ to output a Graphviz representation of a
your binary tree "

In [59]:
class GraphvizNodeBinaryTree(NodeBinaryTree):
    """Abstract class to represent the a Binary Tree of Node with outputs to Graphviz."""
    def __str__(self):
        """Return the DOT language connections of the tree graph in Graphviz"""
        graphviz_code = ''
        
        aux_arr = []
        aux_arr.append(self.root())
        
        for n in aux_arr:
            if n._left is not None:
                graphviz_code += f'  {n._element} -- {n._left._element};\n'
                aux_arr.append(n._left)
            if n._right is not None:
                graphviz_code += f'  {n._element} -- {n._right._element};\n'
                aux_arr.append(n._right)

        return graphviz_code
    
    def __repr__(self):
        """Return the DOT language code to create the tree graph in Graphviz"""
        graphviz_code = ''

        graphviz_code += 'graph node_binary_tree {\n'
        graphviz_code += '  node [shape=circle]\n'
        
        aux_arr = []
        aux_arr.append(self.root())
        
        for n in aux_arr:
            if n._left is not None:
                graphviz_code += f'  {n._element} -- {n._left._element};\n'
                aux_arr.append(n._left)
            if n._right is not None:
                graphviz_code += f'  {n._element} -- {n._right._element};\n'
                aux_arr.append(n._right)
        
        graphviz_code += '}'

        return graphviz_code

In [61]:
graph_binary_tree = GraphvizNodeBinaryTree()
graph_binary_tree._add_root(4)
graph_binary_tree._add_left(graph_binary_tree.root(), 3)
graph_binary_tree._add_right(graph_binary_tree.root(), 5)
graph_binary_tree._add_left(graph_binary_tree.root()._left, 2)
graph_binary_tree._add_right(graph_binary_tree.root()._right, 6)
graph_binary_tree._add_left(graph_binary_tree.root()._left._left, 1)
graph_binary_tree._add_right(graph_binary_tree.root()._right._right, 7)
graph_binary_tree._add_left(graph_binary_tree.root()._right._right, 6.5)
print("----- STR print -----")
print(str(graph_binary_tree))
print("----- REPR print -----")
print(repr(graph_binary_tree))

----- STR print -----
  4 -- 3;
  4 -- 5;
  3 -- 2;
  5 -- 6;
  2 -- 1;
  6 -- 6.5;
  6 -- 7;

----- REPR print -----
graph node_binary_tree {
  node [shape=circle]
  4 -- 3;
  4 -- 5;
  3 -- 2;
  5 -- 6;
  2 -- 1;
  6 -- 6.5;
  6 -- 7;
}
